In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

C:\Users\abido\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.8.0'

### Loading the data

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Model

In [3]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

In [12]:
## tf.keras.callbacks.ModelCheckpoint is a callback that performs this task. 
# The callback takes a couple of arguments to configure checkpointing.

checkpoint_path = "./training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

os.mkdir("./training_1")

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [13]:
model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 815us/step - loss: 1.2352 - acc: 0.6450 - val_loss: 0.7446 - val_acc: 0.7680

Epoch 00001: saving model to ./training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 449us/step - loss: 0.4344 - acc: 0.8830 - val_loss: 0.5227 - val_acc: 0.8390

Epoch 00002: saving model to ./training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 468us/step - loss: 0.2820 - acc: 0.9270 - val_loss: 0.4999 - val_acc: 0.8490

Epoch 00003: saving model to ./training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 449us/step - loss: 0.2241 - acc: 0.9430 - val_loss: 0.4600 - val_acc: 0.8480

Epoch 00004: saving model to ./training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 1s 616us/step - loss: 0.1687 - acc: 0.9610 - val_loss: 0.4127 - val_acc: 0.8660

Epoch 00005: saving model to ./training_1/cp.ckpt
Epoch 6/10
1000/1

### Untrained Model

In [14]:
## Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained
# model will perform at chance levels (~10% accuracy):

model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 247us/step
Untrained model, accuracy: 10.60%


### Load weights from checkpoint

In [15]:
# Then load the weights from the checkpoint, and re-evaluate:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 73us/step
Restored model, accuracy: 87.30%


## Checkpoint callback options
* Train a new model, and save uniquely named checkpoints once every 5-epochs:

In [16]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

os.mkdir("./training_2")

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


### Now, have a look at the resulting checkpoints (sorting by modification date):

In [17]:
import pathlib

# Sort the checkpoints by modification time.
#checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = pathlib.Path(checkpoint_dir).glob("*.ckpt")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
#latest = str(checkpoints[-1])
latest = repr(checkpoints[-1])[13:-2]
checkpoints


[WindowsPath('training_2/cp-0005'),
 WindowsPath('training_2/cp-0010'),
 WindowsPath('training_2/cp-0015'),
 WindowsPath('training_2/cp-0020'),
 WindowsPath('training_2/cp-0025'),
 WindowsPath('training_2/cp-0030'),
 WindowsPath('training_2/cp-0035'),
 WindowsPath('training_2/cp-0040'),
 WindowsPath('training_2/cp-0045'),
 WindowsPath('training_2/cp-0050')]

### Evaluate the model using the last checkpoint weights

In [18]:
model = create_model()
model.load_weights(latest +".ckpt")
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 154us/step
Restored model, accuracy: 88.00%


## Manually save weights

In [19]:
# Save the weights
model.save_weights('./training_1/my_checkpoint')

In [20]:
# Restore the weights
model = create_model()
model.load_weights('./training_1/my_checkpoint')

In [21]:
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 179us/step
Restored model, accuracy: 88.00%


## Save the entire model

The entire model can be saved to a file that contains the weight values, the model's 
configuration, and even the optimizer's configuration. This allows you to checkpoint a 
model and resume training later—from the exact same state—without access to the original 
code.

Saving a fully-functional model in Keras is very useful—you can load them in 
TensorFlow.js and then train and run them in web browsers.

Keras provides a basic save format using the HDF5 standard. For our purposes, the saved 
model can be treated as a single binary blob

In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 690us/step - loss: 1.1649 - acc: 0.6860
Epoch 2/5
1000/1000 [==============================] - 0s 384us/step - loss: 0.4291 - acc: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 330us/step - loss: 0.2751 - acc: 0.9290
Epoch 4/5
1000/1000 [==============================] - 1s 561us/step - loss: 0.2000 - acc: 0.9590 0s - loss: 0.1990 - acc:
Epoch 5/5
1000/1000 [==============================] - 0s 352us/step - loss: 0.1520 - acc: 0.9720


In [23]:
# Save entire model to a HDF5 file
model.save('my_model.h5')

In [24]:
## Now recreate the model from that file:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 203us/step
Restored model, accuracy: 87.80%


Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will loose the state of the optimizer.
